# 面向对象高级编程

## 使用\_\_slots\_\_  
用于限制class能添加的属性

In [46]:
class Student(object):
    pass

In [47]:
s = Student()

In [48]:
s.name = 'Michael'  #动态给实例绑定一个属性
print(s.name)

Michael


In [49]:
def set_age(self,age): #定义一个函数作为实例方法
    self.age = age

In [52]:
from types import MethodType
s.set_age = MethodType(set_age,s) #给实例绑定一个方法
s.set_age(25) #调用实例方法
s.age #测试结果

25

In [21]:
#但是，给一个实例绑定的方法，对另一个实例是不起作用的：
s2 = Student() # 创建新的实例
>>> s2.set_age(25) # 尝试调用方法

SyntaxError: invalid syntax (<ipython-input-21-8ee8c96b2ae5>, line 3)

为了给所有实例都绑定方法，可以给class绑定方法：

In [53]:
def set_score(self,score):
    self.score = score
    
Student.set_score = set_score #不是绑定方法，仅仅是函数调用

s.set_score(100)
s.score

100

In [55]:
from types import MethodType

def set_age(self, age):
    self.age = age

Student.set_age = MethodType(set_age, Student)  #給类绑定方法
s = Student()
s.set_age(25)
s.age

25

100

In [23]:
s2 = Student() 
s2.set_score(99)
s2.score

99

动态语言允许在程序运行的过程中动态给class加上功能，而静态语言很难实现

如果我们想要限制实例的属性怎么办？比如只允许对Student实例添加name和age属性  
定义\_\_slots\_\_变量限制class能添加的属性

In [24]:
class Student(object):
    __slots__ = ('name','age')

In [26]:
s = Student()
s.name = 'Michael'
s.age = 25
s.score = 99 #由于score不在__slots__中

AttributeError: 'Student' object has no attribute 'score'

\_\_slots\_\_仅对当前类实例起作用，对继承的子类是不起作用的

In [27]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 100 #可以添加score属性！

除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__。

In [28]:
class Student(object):
    __slots__ = ('name','age','score')


class GraduateStudent(Student):
    __slots__ = ('deadline')

In [30]:
g1 = GraduateStudent
g1.score = 100
g1.score

100

## 使用@property

In [56]:
s =Student()
s.score = 9999 #我们可以随便更改参数score,因为属性是暴露的

In [58]:
class Student(object):
    def get_score(self): #设置成绩
        return self._score
    def set_score(self,value): #获取成绩
        if not isinstance(value,int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0-100!')
        self._score = value

In [64]:
s = Student()
s.set_score(60)
s.get_score()

60

以上调用方法的缺点：调用复杂

Python内置的\@property装饰器就是负责把一个方法变成属性调用的：

**详细讲解property：**

可能对这段代码中property毫无头绪，那就去查查官网文件吧    
https://docs.python.org/3.6/library/functions.html?highlight=property#property  

class property(fget=None, fset=None, fdel=None, doc=None)  
    return a property attribute #返回一个property的属性  

fget: a function for getting an attribute value #获取属性     
fset: a function for setting an attribute value #设置属性   
fdel: a function for deleting an attribute value #删除属性   
doc: creates a docstring for the attribute  #用于解释语句  

    class C:
        def __init__(self):
            self._x = None
        def getx(self):
            return self._x
        def setx(self, value):
            self._x = value
        def delx(self):
            del self._x

        x = property(getx, setx, delx, "I'm the 'x' property.")  

如果c是C的一个实例，c.x会调用getter，c.x=value会调用setter，del c.x调用deleter。  
如果fget存在的话，doc在没有输入的情况下复制fget的docstring，有利于创建只读属性.

    class Parrot:
        def __init__(self):
            self._voltage = 100000

        @property
        def voltage(self):
            """Get the current voltage."""
            return self._voltage  
        
@property装饰器将voltage()方法变成“getter”属性，在这里是一个只读属性，直接复制voltage的docstring “Get the current voltage.”。  

A property object has getter, setter, and deleter methods usable as decorators that create a copy of the property with the corresponding accessor function set to the decorated function.  
property对象有getter,setter和deleter三种方法，这些方法可以用作装饰器，将访问函数设置成已装饰函数。(翻译待定...)

简单例子：  

    class C:
        def __init__(self):
            self._x = None

        @property
        def x(self):
            """I'm the 'x' property."""
            return self._x

        @x.setter
        def x(self, value):
            self._x = value

        @x.deleter
        def x(self):
            del self._x

In [ ]:
class Student(object):
    @property
    def score(self):
        return self._score
    
    @score.setter #xxx.setter的xxx与property下的方法名字一样，例如这里为score
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0-100')
        self._score = value

In [66]:
s = Student()
s.score = 60 #实际转化为s.set_score=60
s.score

60

定义只读属性：只定义getter方法，不定义setter方法

In [ ]:
class Student(object):
    
    @property
    def birth(self):  #可读写属性
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth = value
    
    @property
    def age(self):  #只读属性
        return 2015 - self.birth

练习：请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution

In [ ]:
class Screen(object):
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self,value):
        self._width = value
        
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self,value):
        self._height = value
        
    @property
    def resolution(self):
        return self._width * self._height

## 多重继承

举例：  
Dog - 狗 - 哺乳动物 - 能跑  
Bat - 蝙蝠 - 哺乳动物 - 能飞  
Parrot - 鹦鹉 - 鸟类 - 能飞  
Ostrich - 鸵鸟 - 鸟类 - 能跑  

如果我们想综合动物分类和能力分类，则我们必须使用多重继承

1. 先设定各类的层级关系

In [2]:
class Animal(object):
    pass

#大类
class Mammal(Animal):  #哺乳动物
    pass

class Bird(Animal):  #鸟类
    pass

#各种动物
class Dog(Mammal):
    pass
class Bat(Mammal):
    pass
class Parrot(Bird):
    pass
class Ostrich(Bird):
    pass

2.定义能跑(Runnale)和能飞(Flyable)的功能

In [3]:
class Runnable(object):
    def run(self):
        print("Running...")
        
class Flyable(object):
    def fly(self):
        print("Flying")

3.给动物加上功能的继承

In [4]:
class Dog(Mammal,Runnable):
    pass
class Bat(Mammal,Flyable):
    pass
class Parrot(Bird,Flyable):
    pass
class Ostrich(Bird,Runnable):
    pass

### Mixln

在设计类的继承关系时，通常，主线都是单一继承下来的，例如，Ostrich继承自Bird。但是，如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让Ostrich除了继承自Bird外，再同时继承Runnable。这种多重继承的设计通常称之为MixIn。

在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

应用举例：  
网络服务：TCPServer和UDPServer  
多进程或多线程模型： ForkingMixIn和ThreadingMixIn  
通过组合，就可以提供合适的服务，比如class MyTCPServer(TCPServer,ForkingMixIn) #多进程模式的TCP服务

Summary：  
- 由于Python允许使用多重继承，因此，MixIn就是一种常见的设计。
- 只允许单一继承的语言（如Java）不能使用MixIn的设计

## 定制类

Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。比如\_\_slots\_\_用于限制class的属性，\_\_len()\_\_为了让class作用于len()函数。

### \_\_str\_\_

In [6]:
class Student(object):
    def __init__(self,name):
        self.name = name
print(Student('Michael'))

返回：<__main__.Student object at 0x000001E0384B9E10>，很差的可读性

In [9]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object (name:%s)'%self.name
    
print(Student('Michael'))

Student object (name:Michael)


返回：Student object (name:Michael)，容易看出来实例内部的重要数据

In [10]:
s = Student('Michael')  #定义实例
s

返回：<__main__.Student at 0x1e03854c748>，不是Student object (name:Michael)  
为什么？  
没有print调用的不是\_\_str\_\_(),而是\_\_repr\_\_(),两者的区别是\_\_str\_\_()返回用户看到的字符串，而\_\_repr\_\_返回程序开发者看到的字符串，即为调试而服务的。  
解决方法？  
再定义一个\_\_repr\_\_(),内容与\_\_str\_\_()完全一样 -> 简便方法：\_\_repr\_\_ = \_\_str\_\_

In [14]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' %self.name
    __repr__ = __str__

In [15]:
s = Student('Michael')  
s

Student object (name=Michael)

### \_\_iter\_\_

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个\_\_ite\_\_()方法，该方法返回一个**迭代对象**，然后，Python的for循环就会不断调用该迭代对象的\_\_next\_\_()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

举例：斐波那契数列

In [16]:
class Fib(object):
    def __init__(self):
        self.a,self.b = 0,1 #初始值a=0,b=1
    def __iter__(self):
        return self  #实例本身就是迭代对象，故返回自己
    def __next__(self):
        self.a,self.b = self.b,self.a+self.b
        if self.a > 100: #退出循环条件
            raise StopIteration()
        return self.a #返回下一个值

In [17]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89


Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，取第5个元素：

In [18]:
Fib()[5]

TypeError: 'Fib' object does not support indexing

解决方法： \_\_gentim\_\_

### \_\_getitem\_\_ ， \_\_setitem\_\_ ， \_\_delitem\_\_ 

In [26]:
class Fib(object):
    def __getitem__(self,n): #用于出现索引的计算中，比如f[i]
        a,b = 1,1
        for x in range(n):
            a,b = b,a+b
        return a

In [27]:
f = Fib()

In [28]:
f[0]

1

In [29]:
#list的切片方法
list(range(100))[5:10]

[5, 6, 7, 8, 9]

In [30]:
f[5:10]

TypeError: 'slice' object cannot be interpreted as an integer

f[5:10]报错，是因为\_\_getitem\_\_()传入的参数可能是整数int，也可能是切片对象slice，我们需要做出判断

In [31]:
class Fib(object):
    def __getitem__(self,n):
        
        if isinstance(n,int):    #判断n是否为整数
            a,b = 1,1
            for x in range(n):
                a,b = b,a+b
            return a
        
        if isinstance(n,slice):  #判断n是否为切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a,b = 1,1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a,b = b,a+b
            return L
            

In [32]:
f = Fib()
f[0:5]

[1, 1, 2, 3, 5]

In [33]:
f[0:5:2] #无法对step做出处理

[1, 1, 2, 3, 5]

当实例通过 a[index] 这种方式调用的时候，会触发\_\_getitem\_\_方法; a[index]=value 会触发\_\_setitem\_\_方法; del a[index]会触发\_\_delitem\_\_方法。

In [43]:
class Tag:
    def __init__(self):
        self.change={'python':'This is python',
                     'php':'PHP is a good language'}
 
    def __getitem__(self, item):
        print('调用getitem')
        return self.change[item]
 
    def __setitem__(self, key, value):
        print('调用setitem')
        self.change[key]=value
 
    def __delitem__(self, key):
        print('调用delitem')
        del self.change[key]
 
a=Tag()

In [44]:
a['php']

调用getitem


'PHP is a good language'

In [45]:
del a['php'] #删除'php':'PHP is a good language'
a.change

调用delitem


{'python': 'This is python'}

In [46]:
a['python'] = 'Changed'
a.change

调用setitem


{'python': 'Changed'}

### \_\_getattr\_\_

调用不存在的属性时，\_\_getattr\_\_动态返回一个属性。

In [47]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'

In [48]:
s = Student()
s.score #没有score属性，因此报错

AttributeError: 'Student' object has no attribute 'score'

In [49]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
        
    def __getattr__(self,attr):
        if attr == 'score':
            return 99 #返回整数

In [50]:
s = Student()
s.score

99

In [63]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
        
    def __getattr__(self,attr):
        if attr == 'score':
            return lambda:99 #返回函数
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)  #其他属性抛出错误提示

In [64]:
s = Student()
s.score() #函数调用

99

In [66]:
s.age #若没有错误提示raise,任意调用某个属性会返回None

AttributeError: 'Student' object has no attribute 'age'

优点： 可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

举个例子：  
现在很多网站都搞REST API，比如新浪微博、豆瓣啥的，调用API的URL类似：
- http://api.server/user/friends
- http://api.server/user/timeline/list  

利用完全动态的\_\_getattr\_\_，我们可以写出一个链式调用：

In [79]:
class Chain(object):
    def __init__(self,path=''):
        self._path = path
        
    def __getattr__(self,path):
        return Chain('%s/%s' %(self._path,path))
    
    def __str__(self):
        return self._path
    
    __repr__ = __str__

In [80]:
Chain().status.user.timeline

/status/user/timeline

升级版：  
还有些REST API会把参数放到URL中，比如GitHub的API： GET /users/:user/repos  
：user需要替换为实际的用户名，比如 Chain().users('michael').repos

In [10]:
#方法一： __call__()，优点在于即使不是users，换成其他的也可以运行，不用改代码
class Chain:
    def __init__(self, path=''):
        #print("1: ",path)
        self._path = path
    def __getattr__(self, path):
        #print("2: ",path)
        return Chain('%s/%s' % (self._path, path))
    def __call__(self, username): #通过users('Michael')调用__call__
        #print("3: ",username)
        return Chain('%s/%s' % (self._path, username))
    def __str__(self):
        #print("4")
        return self._path
    __repr__ = __str__


In [11]:
Chain().users('Michael').repos

/users/Michael/repos

In [16]:
#方法二: 定义一个users方法
class Chain:
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    def users(self, username): #通过定义users属性直接拼接，缺点是如果换成其他名这个就要不停地改
        return Chain('%s/%s%s' % (self._path, 'users/',username))
    def __str__(self):
        return self._path
    __repr__ = __str__

In [17]:
Chain().users('Michael').repos

/users/Michael/repos

In [26]:
#方法三：在__getattr__中添加条件语句，缺点跟方法二一样
class Chain:
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, path):
        if path == 'users':
            return lambda name: Chain('%s/%s/%s' % (self._path, path, name))
        return Chain('%s/%s' % (self._path, path))
    def __str__(self):
        return self._path
    __repr__ = __str__

In [27]:
Chain().users('Michael').repos

/users/Michael/repos

### \_\_call\_\_  
任何类，只需要定义一个\_\_call\_\_()方法，就可以直接对实例进行调用。

In [30]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __call__(self):
        print('My name is %s' % self.name) 

In [31]:
s = Student('Michael')
s()  #函数调用方式

My name is Michael


如何判断一个对象是否能被调用呢？  
Callable

In [33]:
callable(Student('Michael'))  #可调用

True

In [34]:
class Student(object): 
    def __init__(self,name):
        self.name = name

In [35]:
callable(Student('Michael')) #不可调用

False

本节介绍的是最常用的几个定制方法，还有很多可定制的方法，请参考Python官方文档：  
https://docs.python.org/3/reference/datamodel.html#special-method-names

## 使用枚举类

定义月份的时候，我们可以用大写变量通过整数来定义，但是缺点是这些常量仍然是变量,隐患是可以被修改。  
更好的方法是为这样的枚举类型定义一个class类型，然后每个常量都是class的一个唯一实例。(Enum类)

In [36]:
from enum import Enum

In [37]:
Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [38]:
for name, member in Month.__members__.items():  #枚举Month的所有成员
    print(name, '=>', member, ',', member.value) #value属性则是自动赋给成员的int常量，默认从1开始计数

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


In [39]:
#如果需要更精确地控制枚举类型，可以从Enum派生出自定义类
from enum import Enum, unique

@unique  #@unique装饰器可以帮助我们检查保证没有重复值。
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [47]:
print(Weekday.Sun)
print(Weekday.Mon.name) #成员名称
print(Weekday.Mon.value) #用value获得常量
print(Weekday['Mon']) #通过成员名来获取成员
print(Weekday(1))  #通过成员值来获取成员
print(repr(Weekday.Mon))

Weekday.Sun
Mon
1
Weekday.Mon
Weekday.Mon
<Weekday.Mon: 1>


补充材料：  
https://segmentfault.com/a/1190000017327003

练习：把Student的gender属性改造为枚举类型，可以避免使用字符串：

In [48]:
from enum import Enum,unique

@unique
class Gender(Enum):
    Male = 0
    Female =1
    
class Student(object):
    def __init__(self,name,gender):
        self.name = name
        self.gender = gender
        

In [49]:
bart = Student('Bart', Gender.Male)

In [50]:
bart.gender

<Gender.Male: 0>

## 使用元类

### type()

type()函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过type()函数创建出Hello类，而无需通过class Hello(object)...的定义

要创建一个class对象，type()函数依次传入3个参数：  
- class的名称；
- 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
- class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

In [52]:
def fn(self,name='world'):  #定义函数，name可改
    print('Hello, %s.' % name)
    
Hello = type('Hello',(object,),dict(hello=fn))  #创建Hello class

In [53]:
h = Hello()
h.hello()

Hello, world.


In [54]:
h.hello('Michael')

Hello, Michael.


In [55]:
print(type(Hello))

<class 'type'>


type()函数也允许我们动态创建出类

### metaclass  
metaclass用于控制类的创建行为，允许创建类或者修改类  
创建顺序: metaclass - 类 - 实例

In [57]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)  #新增add方法
        return type.__new__(cls, name, bases, attrs)

__new__()方法接收到的参数依次是：
- 当前准备创建的类的对象；
- 类的名字；
- 类继承的父类集合；
- 类的方法集合。

In [58]:
#当传入metaclass关键字时，指示python解释器创建Mylist是需要通过ListMetaclass.__new__()来创建
class MyList(list, metaclass=ListMetaclass):  
    pass

In [60]:
# 测试是否可以调用add方法
L=MyList()
L.add(1)
L

[1]

通常情况下,我们可以直接在MyList中定义，这样更简单。 但是ORM就是一个需要通过metaclass修改类定义的典型例子。 ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。  
要编写一个ORM框架，所有的类都只能动态定义，因为只有使用者才能根据表的结构定义出对应的类来。

**尝试编写一个ORM框架：**

<font color='red'>这个例子不是很理解，往后再来看！！！</font>

1. 写调用接口。比如，使用者如果使用这个ORM框架，想定义一个User类来操作对应的数据库表User

In [69]:
class User(Model):  #父类Model和属性类型StringField、IntegerField是由ORM框架提供的
    #定义类的属性到列的映射
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')
    
#创建一个实例
u = User(id=12345,name='Michael',email='test@orm.org',password='my-pwd')
#保存到数据库
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


2. 定义Field类，负责保存数据库表的字段名和字段类型

In [65]:
class Field(object):
    def __init__(self,name,column_type):  #name-字段名， column_type-字段类型
        self.name = name
        self.column_type = column_type
        
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__,self.name)

3. 定义StringField，IntegerField等，这些都继承Field

In [66]:
class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

super() 函数是用于调用父类(超类)的一个方法,解决多重继承问题的。  
super(type[, object-or-type])  
type - 类  
object-or-type - 类，一般是self

4. 编写最复杂的ModelMetaclass

In [67]:
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)

5. 基类Model

In [68]:
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))